In [77]:
import numpy as np
import sympy as sp
import functions as fun
from IPython.display import display, Math
import scipy.integrate as integ
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import math

hbar = 1
me   = 1
mp = 1836.15
u = (me*mp)/(me+mp)
u=1
Z = 1
alfa = 1/137
c = 1/alfa

chi_alf = np.array([1,0])
chi_bet = np.array([0,1])

w,r, phi, theta , phi_p,theta_p = sp.symbols('w r phi theta phi_p theta_p', real=True)

dtheta = sp.lambdify(theta,sp.sin(theta),"numpy")


hart2cm = 455.633 # en angstroms
hart2ev = 27.2116 # en electronvoltios
hart2invcm = 219475

"""
Importamos las funciones definidas del anterior taller para asi usar los autoestados del atomo de hidrogeno
"""


def eferic_armonics_numeric(var1,var2,l,m):
    ef = (sp.sqrt((2*l+1)/(4*sp.pi) * sp.factorial(l-abs(m))/sp.factorial(l+abs(m)) )
          * sp.exp(sp.I * m * var2) * 1/(2**l * sp.factorial(l)) *
          (1-w**2)**(abs(m)/2)* sp.diff((w**2-1)**l,(w,l+abs(m))))
    ef = ef.subs(w,sp.cos(var1))
    return ef

def chi(s,ms):
    if ms == 1/2:
        return np.array([1,0])
    elif ms == -1/2:
        return np.array([0,1])

def chi_finder(f):
    try :
        if float(np.dot(chi_alf,f)) == 0: # significa que la función de onda tiene como ms = -1/2,  osea, f = [0,psi]
            return chi_bet
    except TypeError:
        # si llega hasta aca es porque capto que el producto punto es una función y no cero, lo que indica que la entrada es una función de onda con spin ms = +1/2
        # osea f = [psi,0],
        return chi_alf


def Radial_part_numeric(var1,n,l):
    p = 2*Z*u/n  * var1
    radial = sp.S(0)
    for k in range(n-l):
        radial +=  (-1)**(k+1) * (sp.factorial(n+l))**2/ (sp.factorial(n-l-1-k)*sp.factorial(2*l+1+k)*sp.factorial(k)) * p**k

    radial *= - sp.sqrt(((2*Z*u/n)**3 * sp.factorial(n-l-1)/(2*n*(sp.factorial(n+l))**3  ))) * sp.exp(-p/2) * p**l

    return radial


def integrate_2d(Z, x, y, method):
    """
    Z: 2D array shaped (len(y), len(x))  -> f(y_i, x_j)
    x: 1D array (len Nx)
    y: 1D array (len Ny)
    method: 'rect', 'trapz', or 'simps'
    """
    if method == 'rect':
        dx = x[1]-x[0]
        dy = y[1]-y[0]
        return np.sum(Z) * dx * dy
    elif method == 'trapz':
        return np.trapezoid(np.trapezoid(Z, x=x, axis=1), x=y, axis=0)
    elif method == 'simps':
        return integ.simpson(integ.simpson(Z, x=x, axis=1), x=y, axis=0)
    else:
        raise ValueError("method must be 'rect','trapz' or 'simps'")


def safe_lambdify(vars, expr):
    f = sp.lambdify(vars, expr, "numpy")
    return lambda *args: np.broadcast_to(f(*args), np.broadcast(*args).shape)

def Psi(var1,var2,var3,n,l,ml,s,ms):
    psi = Radial_part_numeric(var1,n,l)*eferic_armonics_numeric(var2,var3,l,ml)*chi(s,ms)
    return psi


def L2_op (f):
    L2 = -hbar**2*(sp.diff(f,theta,2)   +  sp.cot(theta)*sp.diff(f,theta)+1/(sp.sin(theta))**2 *sp.diff(f,phi,2))
    return L2

def R_op(f):
    rop = -hbar**2/(2*u)* (sp.diff(f,r,2) + 2/r * sp.diff(f,r))
    return rop

def Hamiltonian(f):
    Ham = R_op(f) + 1/(2*u*r**2)*L2_op(f) -Z/r * f
    return Ham

def Matx_element_Op (f,op):
    # this finds the matrix element of an specific operator
    return sp.integrate(sp.integrate(sp.integrate(np.dot(sp.conjugate(f),op)*r**2*sp.sin(theta),(r,0,sp.oo)),(theta,0,sp.pi)),(phi,0,2*sp.pi))

def quantum_nums(f):

    ml = round(float(Matx_element_Op(f,Lz_op(theta,phi,np.dot(chi_alf+chi_bet,f))*chi_finder(f)  )))

    l_l_1 = float(Matx_element_Op(f,L2_op(np.dot(chi_alf+chi_bet,f))*chi_finder(f)    ))
    l_var = sp.symbols('l_var')
    y = l_var*(l_var+1) - l_l_1
    sol = sp.solve(y,l_var)
    l = round(max(sol))

    En = float(Matx_element_Op(f,Hamiltonian(np.dot(chi_alf+chi_bet,f))*chi_finder(f)   ))
    n = round(math.sqrt(-u*Z**2/(2*En)))

    s = 1/2

    chi = chi_finder(f)
    ms = 0
    if chi[0] == chi_alf[0]:
        ms = 1/2
    elif chi[1] == chi_bet[1]:
        ms = -1/2

    return [n,l,ml,s,ms]



def J2_op (f):
    j2 = L2_op(theta,phi,f) + L2_op(theta_p,phi_p,f) + 2*(Lx_op(theta,phi,f)+Ly_op(theta,phi,f)+Lz_op(theta,phi,f))* (Lx_op(theta_p,phi_p,f)+Ly_op(theta_p,phi_p,f)+Lz_op(theta_p,phi_p,f))
    return j2

def Jz_op (f):
    jz = Lz_op(theta,phi,f) + Lz_op(theta_p,phi_p,f)
    return jz


def clebseth_G(j1,j2,m1,m2,j,m,Sp_notSp_):
    if Sp_notSp_ == True:
        return sp.integrate(sp.integrate( np.dot(sp.conjugate(Y_S(j1,m1,j2,m2)) , eferic_armonics_numeric(theta,phi,j,m)* chi_finder(Y_S(j1,m1,j2,m2))  ) * sp.sin(theta),(theta,0,sp.pi)),(phi,0,2*sp.pi))
    else:
        return abs(sp.integrate(sp.integrate( sp.conjugate(Y1_Y2(j1,m1,j2,m2)) * eferic_armonics_numeric(theta,phi,j,m) *sp.sin(theta),(theta,0,sp.pi)),(phi,0,2*sp.pi)))

def Y1_Y2  (l,ml,lp,mp):
    mult_esferic = eferic_armonics_numeric(theta,phi,l,ml) * eferic_armonics_numeric(theta_p,phi_p,lp,mp)
    return mult_esferic

def Y_S (l,ml,s,ms):
    mult_esferic = eferic_armonics_numeric(theta,phi,l,ml) * chi(s,ms)
    return mult_esferic


def Lx_op (var1,var2,f):
    Lx = 1/2 * (Lplus_op(var1,var2,f)+Lminus_op(var1,var2,f))
    return Lx

def Ly_op (var1,var2,f):
    Ly= 1/(2*sp.I) * (Lplus_op(var1,var2,f)-Lminus_op(var1,var2,f))
    return Ly

def Lz_op (var1,var2,f):
    Lz = -sp.I*hbar*sp.diff(f,var2)
    return Lz

def Lplus_op(var1,var2,f):
    try :
        if float(np.sum(f)) == 0:
            return np.array([0,0])

    except TypeError:
        qnums = quantum_nums(f)
        l_l_1 = float(Matx_element_Op(f,L2_op(np.dot(chi_alf+chi_bet,f))*chi_finder(f)    ))

        if qnums[2] == qnums[1] :
            return np.array([0,0])
        else:
            return hbar* math.sqrt(l_l_1-qnums[2]*(qnums[2]+1)) *Psi(r,var1,var2,qnums[0],qnums[1],qnums[2]+1,qnums[3],qnums[4])

def Lminus_op(var1,var2,f):
    try :
        if float(np.sum(f)) == 0:
            return np.array([0,0])

    except TypeError:
        qnums = quantum_nums(f)
        l_l_1 = float(Matx_element_Op(f,L2_op(np.dot(chi_alf+chi_bet,f))*chi_finder(f)    ))

        if qnums[2] == -qnums[1] :
            return np.array([0,0])
        else:
            return hbar* math.sqrt(l_l_1-qnums[2]*(qnums[2]-1)) *Psi(r,var1,var2,qnums[0],qnums[1],qnums[2]-1,qnums[3],qnums[4])


def Splus_op(f):
    try :
        if float(np.dot(chi_alf,f)) == 0: # significa que la función de onda tiene como ms = -1/2,  osea, f = [0,psi]
            return hbar * np.array([f[1],0])
    except TypeError:
        # si llega hasta aca es porque capto que el producto punto es una función y no cero, lo que indica que la entrada es una función de onda con spin ms = +1/2
        # osea f = [psi,0], S+ deberia de anular esto.
        return f[0] * np.array([0,0])

def Sminus_op(f):
    try :
        if float(np.dot(chi_alf,f)) == 0: # significa que la función de onda tiene como ms = -1/2,  osea, f = [0,psi] , el resultado seria anular
            return f[1] * np.array([0,0])
    except TypeError:
        # si llega hasta aca es porque capto que el producto punto es una función y no cero, lo que indica que la entrada es una función de onda con spin ms = +1/2
        # osea f = [psi,0]
        return hbar * np.array([0,f[0]])

def S2_op (f):
    return  3/4 * hbar**2 * f

def Sx_op (f):
    sx = 1/2 * (Splus_op(f)+Sminus_op(f))
    return sx

def Sy_op (f):
    sy = 1/(sp.I*2) * (Splus_op(f)-Sminus_op(f))
    return sy

def Sz_op (f):
    try :
        if float(np.dot(chi_alf,f)) == 0: # significa que la función de onda tiene como ms = -1/2,  osea, f = [0,psi]
            return -1/2 * hbar * f
    except TypeError:
        # si llega hasta aca es porque capto que el producto punto es una función y no cero, lo que indica que la entrada es una función de onda con spin ms = +1/2
        # osea f = [psi,0],
        return 1/2 * hbar * f


"""
para comprobar el teorema del virial (2<T> = -<V>) para el caso del atomo de hidrogeno, calcularemos las componentes de matriz de la energia cinetica y potencial.
"""

#Comenzamos definiendo los elementos de matriz usando los autoestados del atomo de hidrogeno
#pero comenzando definiendo los operadores energia cinetica y potencial a partir del hamiltoniano.

def T(f):
    kine = Hamiltonian(f)- (-Z/r * f)
    return kine

def V(f):
    V = Hamiltonian(f)- T(f)
    return V


for j in range(1,10):
    display (Math(r"<\Psi"+f"_{{{j}{0}{0}}}|"+r"\hat{T}|\Psi"+f"_{{{j}{0}{0}}}> =" + sp.latex(round(sp.integrate(sp.integrate(sp.integrate(sp.conjugate(np.dot(chi_alf,Psi(r,theta,phi,j,0,0,1/2,1/2)))*T(np.dot(chi_alf,Psi(r,theta,phi,j,0,0,1/2,1/2)))*r**2*sp.sin(theta),(r,0,sp.oo)),(theta,0,sp.pi)),(phi,0,2*sp.pi))*hart2ev,2)   )  + f"---> "
                                                                                                                                                                                                                                                                                              f"2*{round(sp.integrate(sp.integrate(sp.integrate(sp.conjugate(np.dot(chi_alf,Psi(r,theta,phi,j,0,0,1/2,1/2))    )*T(np.dot(chi_alf,Psi(r,theta,phi,j,0,0,1/2,1/2)))*r**2*sp.sin(theta),(r,0,sp.oo)),(theta,0,sp.pi)),(phi,0,2*sp.pi))*hart2ev,2)} = "
                                                                                                                                                                                                                                                                                              f"{2*round(sp.integrate(sp.integrate(sp.integrate(sp.conjugate(np.dot(chi_alf,Psi(r,theta,phi,j,0,0,1/2,1/2)))*T(np.dot(chi_alf,Psi(r,theta,phi,j,0,0,1/2,1/2)))*r**2*sp.sin(theta),(r,0,sp.oo)),(theta,0,sp.pi)),(phi,0,2*sp.pi))*hart2ev,2)}")  )

print("\n")

for j in range(1,10):
    display (Math(r"<\Psi"+f"_{{{j}{0}{0}}}|"+r"\hat{V}|\Psi"+f"_{{{j}{0}{0}}}> =" + sp.latex(round(sp.integrate(sp.integrate(sp.integrate(sp.conjugate(np.dot(chi_alf,Psi(r,theta,phi,j,0,0,1/2,1/2)))*V(np.dot(chi_alf,Psi(r,theta,phi,j,0,0,1/2,1/2)))*r**2*sp.sin(theta),(r,0,sp.oo)),(theta,0,sp.pi)),(phi,0,2*sp.pi))*hart2ev,2)   ) + f"---> "
                                                                                                                                                                                                                                                                                              f"(-1)*({round(sp.integrate(sp.integrate(sp.integrate(sp.conjugate(np.dot(chi_alf,Psi(r,theta,phi,j,0,0,1/2,1/2)))*V(np.dot(chi_alf,Psi(r,theta,phi,j,0,0,1/2,1/2)))*r**2*sp.sin(theta),(r,0,sp.oo)),(theta,0,sp.pi)),(phi,0,2*sp.pi))*hart2ev,2)}) = "
                                                                                                                                                                                                                                                                                              f"({(-1)*round(sp.integrate(sp.integrate(sp.integrate(sp.conjugate(np.dot(chi_alf,Psi(r,theta,phi,j,0,0,1/2,1/2)))*V(np.dot(chi_alf,Psi(r,theta,phi,j,0,0,1/2,1/2)))*r**2*sp.sin(theta),(r,0,sp.oo)),(theta,0,sp.pi)),(phi,0,2*sp.pi))*hart2ev,2)})" ))




<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [78]:
# Para hallar las correcciones relativistas aplicamos el metodo de perturbaciones, el cual permite hallar energías de la perturbación a partir de las autofunciones del hamiltoniano sin perturbar.

#Para optimizar recursos, ya sabemos que la primera correccion mezcla autoestados de distinto n y de distinto l, por lo que recorreremos una matriz para cada n , una matriz de (n-1)X(n-1),


def T2(f,f_):
    # return Hamiltonian(sp.simplify(Hamiltonian(f) + (Z/r * f))) + Z/r * (Hamiltonian(f) + (Z/r * f))
    return (Hamiltonian(sp.conjugate(f))+ Z/r*sp.conjugate(f))  * (Hamiltonian(f_) + Z/r*f_)

def En (n):
    return -u/2 * (Z**2/n**2)


def MtrxElement_T2(n,l,ml,s,ms,n_,l_,ml_,s_,ms_):
    Op = T2(np.dot(chi_alf,Psi(r,theta,phi,n,l,ml,s,ms)),np.dot(chi_alf,Psi(r,theta,phi,n_,l_,ml_,s_,ms_)))
    return sp.integrate(sp.integrate(sp.integrate( Op *r**2*sp.sin(theta),(r,0,sp.oo)),(theta,0,sp.pi)),(phi,0,2*sp.pi))

num = 3

for n in range(1,num):
    T2matr = np.zeros((n,n))
    for l1 in range(n):
        for l2 in range(l1,n):

            # T2_num = safe_lambdify((theta,phi),sp.integrate(sp.conjugate(Psi(r,theta,phi,n,l1,0))*T2(Psi(r,theta,phi,n,l2,0)),(r,0,sp.oo)))


            T2matr[l1,l2] = round(-1/(2*c**2) * (  MtrxElement_T2(n,l1,0,1/2,1/2,n,l2,0,1/2,1/2)  )* hart2invcm   ,7)

            # T2matr[l1,l2] = np.round(abs(integrate_2d(T2_num(Tvals,Pvals)*dtheta(Tvals),thetvals,phivals,'simps')) ,3) * hart2invcm
            # T2matr[l1,l2] = -1/(2*c**2) * ((En(n))**2 + 2*En(n)*(Z*u/n**2) + Z**2*u**2/(n**3*(l1+1/2)) ) * hart2invcm


    T2matr[1:num,0:num-1] = T2matr[0:num-1,1:num].T
    print(T2matr)


#Comparando los resultados que se obtienen son iguales al resultado analitico:


print("\n")

for n in range(1,num):
    T2matr = np.zeros((n,n))
    for l1 in range(n):
        T2matr[l1,l1] = -1/(2*c**2) * ((En(n))**2 + 2*En(n)*(Z*u/n**2) + Z**2*u**2/(n**3*(l1+1/2)) ) * hart2invcm

    T2matr[1:num,0:num-1] = T2matr[0:num-1,1:num].T
    print(T2matr)

[[-7.3084275]]
[[-1.1876195  0.       ]
 [ 0.        -0.2131625]]


[[-7.30842746]]
[[-1.18761946  0.        ]
 [ 0.         -0.21316247]]


In [98]:
#Ahora haremos lo mismo para el estado n=3 y l = 1 para la correccion relativista spin orbita:

# Definimos la matriz spin orbita como :

def SPO(f):

    spo = 1/2 * alfa**2 * Z/r**3 * (  Lz_op(theta,phi,np.dot(chi_alf+chi_bet,Sz_op(f)))*chi_finder(f)   + 1/2*(Lplus_op(theta,phi,Sminus_op(f)) + Lminus_op(theta,phi,Splus_op(f))    ))

    return spo

#Para n = 3 y l =1 tenemos una matriz 2x2 en la que se acoplan los estados para l-1/2 y l+1/2
n = 3
l = 1

LSmatrix = np.zeros((6,6))
thetvals,phivals = np.linspace(0.00001,np.pi-0.0001,700) , np.linspace(0.00001,2*np.pi-0.00001,700)
Tvals, Pvals = np.meshgrid(thetvals, phivals) # x, y

MS = [-1/2,1/2]
i = 0
j = 0

for m1 in range(-l,l+1):
    for ms1 in range(2):
        for m2 in range(-l,l+1):
            for ms2 in range(2):
                a = sp.integrate(np.dot(sp.conjugate(Psi(r,theta,phi,n,l,m1,1/2,MS[ms1])) , SPO(Psi(r,theta,phi,n,l,m2,1/2,MS[ms2])))*r**2,(r,0,sp.oo))
                b = sp.lambdify((theta,phi),a,'numpy')
                LSmatrix[i,j] = integrate_2d(b(Tvals,Pvals)*dtheta(Tvals),thetvals,phivals,'simps')*hart2invcm
                LSmatrix[i,j] = round(LSmatrix[i,j],5)
                j+=1
        j = 0
        i+= 1


# LSmatrix[1:6,0:6-1] = LSmatrix[0:6-1,1:6].T

eig1, eig2 = np.linalg.eigh(LSmatrix)

print(np.round(eig1,4))



/tmp/ipykernel_3794/1181983583.py:29: ComplexWarning: Casting complex values to real discards the imaginary part
  LSmatrix[i,j] = integrate_2d(b(Tvals,Pvals)*dtheta(Tvals),thetvals,phivals,'simps')*hart2invcm

KeyboardInterrupt



In [97]:
"""
Estudiemos el resultado analitico. de las correcciones de spin orbita.
"""

def SPO_analitic(n,l,ms):

    if ms == 1/2:
        return -En(n)*(Z*alfa)**2/(2*n*l*(l+1/2)*(l+1))  *  l
    elif ms == -1/2:
        return -En(n)*(Z*alfa)**2/(2*n*l*(l+1/2)*(l+1))  *  (-l-1)


SPO_analitic_matrx = np.zeros(2)
i = 0

for ms in range(2):
    SPO_analitic_matrx[i] = round(SPO_analitic(n,l,MS[ms])*hart2invcm,4)
    i += 1


print(SPO_analitic_matrx)

[-0.0722  0.0361]
[-0.07214993 -0.07214993  0.03604993  0.03604993  0.0361      0.0361    ]
[[ 0.          0.         -0.         -0.          1.          0.        ]
 [-0.81659103  0.         -0.57721667 -0.          0.          0.        ]
 [ 0.57721667  0.         -0.81659103 -0.          0.          0.        ]
 [ 0.         -0.57721667 -0.         -0.81659103  0.          0.        ]
 [ 0.          0.81659103 -0.         -0.57721667  0.          0.        ]
 [ 0.          0.         -0.         -0.          0.          1.        ]]
1.0


In [84]:
print(np.round(eig2,4))

eig2 = fun.norm([],eig2)
print("\n")

print(np.round(eig2,4))
"""
Ahora compararemos los coeficientes de Clebset gordan con los autovectores hayados en esta instancia, y demostraremos que usando los autovectores hallados para encontrar los autoestados de J2 y Jz. igualmente funcionará. Usando una tabla de valores de coeficientes de clebseth Gordan.
"""

print("\n")

CLBs_vec = np.zeros(6)
i = 0

jt1 = 1/2
mst1 = [-1/2,1/2]


jt2 = 3/2
mst2 = [-3/2,-1/2,1/2,3/2]


print(CLBs_vec)


[[-0.      0.      0.      0.7071 -0.      0.7071]
 [-0.     -0.8165 -0.4088  0.     -0.4077 -0.    ]
 [ 0.      0.5774 -0.5781  0.     -0.5766 -0.    ]
 [ 0.5774 -0.      0.      0.5774  0.     -0.5773]
 [-0.8165  0.      0.      0.4082  0.     -0.4082]
 [ 0.      0.     -0.7062  0.      0.708   0.    ]]


[[-0.      0.      0.      0.7071 -0.      0.7071]
 [-0.     -0.8165 -0.4088  0.     -0.4077 -0.    ]
 [ 0.      0.5774 -0.5781  0.     -0.5766 -0.    ]
 [ 0.5774 -0.      0.      0.5774  0.     -0.5773]
 [-0.8165  0.      0.      0.4082  0.     -0.4082]
 [ 0.      0.     -0.7062  0.      0.708   0.    ]]


[0. 0. 0. 0. 0. 0.]
